# 1. Introduction
>Just one more kernel with CNN in this competition :)
To work with large data, use ImageDataGenerator.flow_from_dataframe as input for model.
First, import necessary libraries:

In [ ]:
#####################################
# Libraries
#####################################
# Common libs
import pandas as pd
import numpy as np
import sys
import os
import random
import time
from pathlib import Path

# Image processing
import imageio
import PIL
import cv2
import skimage.transform
#from skimage.transform import rescale, resize, downscale_local_mean

# Charts
import matplotlib.pyplot as plt
import seaborn as sns

# ML, statistics
import scipy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score

# Tensorflow
#from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.preprocessing import MultiLabelBinarizer


#############################################
# Settings
#############################################

pd.set_option("display.max_columns", None)

plt.style.use('fivethirtyeight')
#plt.style.use('seaborn')

print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

INPUT_DIR = "../input/plant-pathology-2021-fgvc8/"
PRETRAINED_DIR = "../input/pretrained-model/"
OUTPUT_DIR ="./"

#OUTPUT_DIR = "./model/"

TRAIN_DATA_DIR = INPUT_DIR + 'train_images'
#TRAIN_DATA_DIR = INPUT_DIR + 'img_sz_512'

IMSIZES = (224, 240, 260, 300, 380, 456, 528, 600)

#NUM_CLASSES = 7 #12

NUM_CLASSES = 6 #12
IMAGE_SIZE= (224,224)  #(256,256)

BATCH_SIZE = 32 #32
TRAIN_BATCH_SIZE = 1024 #512

print(os.listdir(OUTPUT_DIR))
print(os.listdir(INPUT_DIR))

In [ ]:
toy=False

all_df = pd.read_csv(INPUT_DIR + "train.csv")
if toy:
    all_df = all_df.sample(5000)


print(all_df.shape)
all_df.head()

In [ ]:
NB_TRAIN = all_df.shape[0]  #15930 #4275 #32706
NB_VALID = NB_TRAIN * 0.1 #1770 #475 #10902

# 2. Exploratory Data Analysis
>Check labels balancing
Check whether labels are balanced

In [ ]:
# Checking if there are any null values in the dataset
all_df.isnull().any()

In [ ]:
# Checking the column data type
all_df.dtypes

In [ ]:
all_df.labels.value_counts().plot(kind='bar')
plt.title('Labels counts')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()
print(all_df.labels.value_counts())

In [ ]:
train = pd.DataFrame()
train['image'] = all_df['image']
train['labels'] = all_df['labels'].apply(lambda string: string.split(' '))
train.head()

In [ ]:
s = list(train['labels'])
mlb = MultiLabelBinarizer()
trainx = pd.DataFrame(mlb.fit_transform(s), columns=mlb.classes_, index=train.index)
print(trainx.columns)
print(trainx.sum())

labels = list(trainx.sum().keys())
print(labels)
label_counts = trainx.sum().values.tolist()

fig, ax = plt.subplots(1,1, figsize=(20,6))

sns.barplot(x= labels, y= label_counts, ax=ax)

        
scab                               1251
healthy                            1209
frog_eye_leaf_spot                  881
rust                                515
complex                             449
powdery_mildew                      320
scab frog_eye_leaf_spot             190
scab frog_eye_leaf_spot complex      53
rust frog_eye_leaf_spot              41
frog_eye_leaf_spot complex           40
rust complex                         29
powdery_mildew complex               22

Labels are unbalanced in train dataset, will figure it out in preprocessing stage.

## View sample images

In [ ]:
fig1 = plt.figure(figsize=(26,20))

for i in range(1, 13):
    
    rand =  random.randrange(1, 18000)
    sample = os.path.join('../input/plant-pathology-2021-fgvc8/train_images/', train['image'][rand])
    
    img = PIL.Image.open(sample)
    
    ax = fig1.add_subplot(4,3,i)
    ax.imshow(img)
    #print(img.size)
    
    title = f"{train['labels'][rand]}{img.size}"
    plt.title(title)
    
    fig1.tight_layout()

# 3. Data preparation
>Here we are going to balance dataset and prepare image generator

## Train and test split

In [ ]:
class DataPreparation:
    """
    Train/test
    """
    def train_test_split(self, all_df):
        """
        Balanced split to train, test and val
        """
        # Split to train and test before balancing
        train_df, test_df = train_test_split(all_df, random_state=24, test_size=0.05, shuffle=False)
        print(train_df.shape)
#         # Split train to train and validation datasets
#         train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=24)
        # Number of samples in each category
        
        #ncat_bal = train_df['labels'].value_counts().max()
#        #ncat_bal = int(len(train_df)/train_df['label'].cat.categories.size)
        #train_df = train_df.groupby('labels', as_index=False).apply(lambda g:  g.sample(ncat_bal, replace=True, random_state=24)).reset_index(drop=True)
        #print(train_df.shape, ncat_bal)
        return train_df, test_df
    
    def plot_balanced(self, train_df, all_df):
        """
        Plot samples per category before and after balancing
        """
        f, axs = plt.subplots(1,2,figsize=(15,4))
        # Before balancing
        all_df.labels.value_counts().plot(kind='bar', ax=axs[0])
        axs[0].set_title('All labels')
        axs[0].set_xlabel('Label')
        axs[0].set_ylabel('Count')
        # After balancing
        train_df.labels.value_counts().plot(kind='bar', ax=axs[1])
        axs[1].set_title('Train labels after balancing')
        axs[1].set_xlabel('Label')
        axs[1].set_ylabel('Count')
        plt.tight_layout()
        plt.show()


# Train/test/validation split with balanced labels in train
data_prep = DataPreparation()
#train_df, test_df = data_prep.train_test_split(all_df)
train_df, test_df = data_prep.train_test_split(train)

# Plot before and after balancing
data_prep.plot_balanced(train_df, all_df)
print(all_df.shape,train_df.shape, test_df.shape)

## Create image generator
>Keras ImageDataGenerator can work with dataframe of file names. Our train, validation and test dataframes contain file name in id column and ImageDataGenerator can understand id.

In [ ]:
class Generators:
    """
    Train, validation and test generators
    """
    def __init__(self, train_df, test_df):
        self.batch_size= BATCH_SIZE #16 #1024 #32 #8
        self.img_size= IMAGE_SIZE #(256,256)#(512,512)#(192,128) #(96,64) (384,256)  #(4000,2672)

        # Base train/validation generator
        _datagen = ImageDataGenerator(
            rescale=1./255.,
            validation_split=0.1,
            brightness_range=(0.8, 1.2),
            #featurewise_center=False,
            #featurewise_std_normalization=False,
            fill_mode='nearest',
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10, #180,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.15, # Randomly zoom image 
            width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=True  # randomly flip images
            )
 
        # Train generator
        self.train_generator = _datagen.flow_from_dataframe(
            dataframe=train_df, #all_df, #
            directory=TRAIN_DATA_DIR,
            x_col="image",
            y_col="labels",
            #has_ext=False,
            
            subset="training",
            batch_size=self.batch_size,
            seed=42,
            shuffle=False, #True,
            class_mode="categorical",
            target_size=self.img_size)
        print('Train generator created')
        # Validation generator
        self.val_generator = _datagen.flow_from_dataframe(
            dataframe=train_df,#test_df ,#
            directory=TRAIN_DATA_DIR,
            x_col="image",
            y_col="labels",
            #has_ext=False,
            
            subset="validation",
            batch_size=self.batch_size,
            seed=42,
            shuffle=False, #True,
            class_mode="categorical",
            target_size=self.img_size)    
        print('Validation generator created')
        
        
        # Test generator
        _test_datagen=ImageDataGenerator(rescale=1./255.)
        self.test_generator = _test_datagen.flow_from_dataframe(
            dataframe=test_df,
            directory=TRAIN_DATA_DIR,
            x_col="image",
            y_col='labels',
            #has_ext=False,
            class_mode="categorical",
            batch_size=self.batch_size,
            seed=42,
            shuffle=False,
            target_size=self.img_size)     
        print('Test generator created')

        
        
        
# Create generators   
#(43608, 2) (4658, 2)
generators = Generators(train_df, test_df)
print("Generators created")

#(18632, 2) (17700, 2) (932, 2)
#15930 , 1770, 932

'''
#https://medium.com/the-owl/k-fold-cross-validation-in-keras-3ec4a3a00538
#K-Fold Cross Validation for Deep Learning Models 
Y = train_df[['labels']]
kf = KFold(n_splits = 5) 
skf = StratifiedKFold(n_split = 5, random_state = 7, shuffle = True) 

n = train_df.shape()[0]
for train_index, val_index in kf.split(np.zeros(n),Y):
    training_data = train_df.iloc[train_index]
    validation_data = train_df.iloc[val_index]
    
    generators = Generators(training_data, validation_data)
'''

In [ ]:
#print(generators.train_generator.next())
train_df.shape

In [ ]:
#print(generators.train_generator.classes)
print(generators.train_generator.class_indices)
class_labels = generators.train_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
print(class_labels)
classes = list(class_labels.values())
print(classes)        
#print(len(self.generator.classes))

#y_pred = []
#for item in self.y_pred:
#    y_pred.append(label_batch)
def get_class(one_hot):
    for i in range(NUM_CLASSES):
        if one_hot[i] == 1:
            return classes[i]
    
            
            
w=10
h=10
fig=plt.figure(figsize=(20, 14))
columns = 2
rows = 4
plt.title('Image Class')
plt.axis('off')
for i in range(1, columns*rows):
    
    img_batch, label_batch = generators.train_generator.next()
    fig.add_subplot(rows, columns, i)
    
    plt.title(get_class(label_batch[i]))  
    plt.imshow(img_batch[i])
    plt.axis('off')
plt.show()

# 4. Create and train the model
>Put all creation and training code in one class. Will experiment with model architecture later.

In [ ]:
LR_START = 0.00001
LR_MAX = 0.0001 
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 15
LR_SUSTAIN_EPOCHS = 3
LR_EXP_DECAY = .8
EPOCHS = 100

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

rng = [i for i in range(EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
class ModelTrainer:
    """
    Create and fit the model
    """
    
    def __init__(self, generators):
        #(18632, 2) (17700, 2) (932, 2)
        #15930 , 1770, 932
        
        self.nb_train_samples = NB_TRAIN  #15930 #4275 #32706
        self.nb_validation_samples = NB_VALID #1770 #475 #10902
        self.batch_size =  TRAIN_BATCH_SIZE #512 #512 #32 #1024   #32
               
        
        self.generators = generators
        self.img_width = IMAGE_SIZE[0]
        self.img_height = IMAGE_SIZE[1]
        print(self.img_width,self.img_height)
        
    
    def create_model(self, model_name):
        #will preload and consume lots memory
        #dict_model = {
        #    "Xception": self.create_model_1(),
        #    "DenseNet121": self.create_model_2(),
        #    "XceptionDenseNet121": self.create_model_3(),
        #    "InceptionResNetV2": self.create_model_4(),
        #    "ResNet50V2": self.create_model_5(),
        #    "VGG16": self.create_model_6(),
        #    "middle": self.create_model_middle(),
        #    "small": self.create_model_small(),
        #}    
        #print("Creating "+model_name)
        if model_name == "Xception" :
                model = self.create_model_1()
        elif model_name == "EfficientNetB7": 
                model = self.create_model_0()                
        elif model_name == "DenseNet121": 
                model = self.create_model_2()
        elif model_name == "XceptionDenseNet121": 
                model = self.create_model_3()
        elif model_name == "InceptionResNetV2": 
                model = self.create_model_4()
        elif model_name == "ResNet50V2": 
                model = self.create_model_5()
        elif model_name == "VGG16": 
                model = self.create_model_6()
        elif model_name == "middle": 
                model = self.create_model_middle()
        elif model_name == "small": 
                model = self.create_model_small()
        else:
                print("Model not found")
        self.model_name = model_name
        return model
    

    def create_model_0(self):
        path0 = PRETRAINED_DIR + 'efficientnetb7_notop.h5'
        pretrained_model = tf.keras.applications.EfficientNetB7(include_top=False, weights=path0, #drop_connect_rate=0.4,
                                                                 input_shape=(self.img_width, self.img_height, 3))
        model = tf.keras.models.Sequential([
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(NUM_CLASSES,activation='sigmoid')
        ])
        
        # Freezing the weights
        for layer in model.layers[:-1]:
            layer.trainable=False
    
        return model    
    
    #Total params: 20,886,068
    def create_model_1(self):
        path1 = PRETRAINED_DIR + 'xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
        pretrained_model = tf.keras.applications.xception.Xception(include_top=False, weights=path1, input_shape=(self.img_width, self.img_height, 3))
        model = tf.keras.models.Sequential([
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(NUM_CLASSES,activation='softmax')
        ])
        return model
    
    def create_model_2(self):
        path2 = PRETRAINED_DIR + 'densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5'
        pretrained_model = tf.keras.applications.densenet.DenseNet121(include_top=False, weights=path2,input_shape=(self.img_width, self.img_height, 3))
        model = tf.keras.models.Sequential([
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(NUM_CLASSES,activation='softmax')
        ])
    
        return model

    def create_model_3(self):
        path1 = PRETRAINED_DIR + 'xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
        xception_model = tf.keras.models.Sequential([
           tf.keras.applications.xception.Xception(include_top=False, weights=path1, input_shape=(self.img_width, self.img_height, 3)),#(512, 512, 3)),
           tf.keras.layers.GlobalAveragePooling2D(),
           #tf.keras.layers.Dense(NUM_CLASSES,activation='softmax')
           tf.keras.layers.Dense(NUM_CLASSES,activation='sigmoid')
        ])
        # Freezing the weights
        #for layer in xception_model.layers[:-1]:
        #    layer.trainable=False
            
        #xception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        #xception_model.summary()
        path2 = PRETRAINED_DIR + 'densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5'
        densenet_model = tf.keras.models.Sequential([
            tf.keras.applications.densenet.DenseNet121(include_top=False, weights=path2,input_shape=(self.img_width, self.img_height, 3)),#(512, 512, 3)),
            tf.keras.layers.GlobalAveragePooling2D(),
            #tf.keras.layers.Dense(NUM_CLASSES,activation='softmax')
            tf.keras.layers.Dense(NUM_CLASSES,activation='sigmoid')
        ])
        #for layer in densenet_model.layers[:-1]:
        #    layer.trainable=False
        #densenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        #densenet_model.summary()
        
        inputs = tf.keras.Input(shape=(self.img_width, self.img_height, 3)) #(512, 512, 3))

        xception_output = xception_model(inputs)
        densenet_output = densenet_model(inputs)

        outputs = tf.keras.layers.average([densenet_output, xception_output])


        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        return model
 
    def create_model_4(self):
        pretrained_model = tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(self.img_width, self.img_height, 3))
        model = tf.keras.models.Sequential([
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(NUM_CLASSES,activation='softmax')
        ])
        return model
    
    def create_model_5(self):
        pretrained_model = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_shape=(self.img_width, self.img_height, 3))
        model = tf.keras.models.Sequential([
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(NUM_CLASSES,activation='softmax')
        ])
    
        return model

    #Total params: 14,720,844
    def create_model_6(self):
        pretrained_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(self.img_width, self.img_height, 3))
        model = tf.keras.models.Sequential([
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(NUM_CLASSES,activation='softmax')
        ])
        return model    
    
    def create_model_middle(self):
        model = Sequential()

        # Padding = 'same'  results in padding the input such that
        # the output has the same length as the original input
        model.add(Conv2D(32, (3, 3), padding='same',
                         input_shape= (self.img_width, self.img_height, 3)))
        model.add(Activation('relu'))
        model.add(Conv2D(32, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(12))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(NUM_CLASSES))
        model.add(Activation('softmax'))

        return model
    
    def create_model_vgg16(self):
        input_shape = (224, 224, 3)

        model = Sequential([
            Conv2D(64, (3, 3), input_shape=input_shape, padding='same',
                   activation='relu'),
            Conv2D(64, (3, 3), activation='relu', padding='same'),
            MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
            Conv2D(128, (3, 3), activation='relu', padding='same'),
            Conv2D(128, (3, 3), activation='relu', padding='same',),
            MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
            Conv2D(256, (3, 3), activation='relu', padding='same',),
            Conv2D(256, (3, 3), activation='relu', padding='same',),
            Conv2D(256, (3, 3), activation='relu', padding='same',),
            MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
            Conv2D(512, (3, 3), activation='relu', padding='same',),
            Conv2D(512, (3, 3), activation='relu', padding='same',),
            Conv2D(512, (3, 3), activation='relu', padding='same',),
            MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
            Conv2D(512, (3, 3), activation='relu', padding='same',),
            Conv2D(512, (3, 3), activation='relu', padding='same',),
            Conv2D(512, (3, 3), activation='relu', padding='same',),
            MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
            Flatten(),
            Dense(4096, activation='relu'),
            Dense(4096, activation='relu'),
            Dense(1000, activation='softmax')
        ])
        return model
    
            
    #Total params: 33,560,364    
    def create_model_small(self):
        """
        Build CNN model using img_width, img_height from fields.
        """
        model=Sequential()
        model.add(Conv2D(16, kernel_size=3, input_shape=(self.img_width, self.img_height,3), activation='relu', padding='same'))
        model.add(MaxPooling2D(2))
        model.add(Conv2D(32, kernel_size=3, activation='relu', padding='same'))
        model.add(Dropout(0.1))
        model.add(Flatten())
        model.add(Dense(64, activation = "relu"))        
        # 1 y label
        model.add(Dense(NUM_CLASSES, activation='softmax'))
        #model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

        # We use a very small learning rate 
        model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])
        return model
    
    def train(self, model, toy):
        """
        Train the model
        """
        if toy:
            epochs=10
            steps_per_epoch=10
            #steps_per_epoch = self.nb_train_samples / self.batch_size
            validation_steps=1
            #validation_steps= self.nb_validation_samples / self.batch_size
        else:
            epochs=50
            #steps_per_epoch=100
            #steps_per_epoch=30 
            
            #steps_per_epoch=20
            steps_per_epoch = self.nb_train_samples / self.batch_size
            
            #steps_per_epoch=10
            #validation_steps=5
            
            #validation_steps=100
            validation_steps= self.nb_validation_samples / self.batch_size


        # val_accuracy should be try and compare with val_loss
        # We'll stop training if no improvement after some epochs
        earlystopper = EarlyStopping(monitor='val_loss', 
                            min_delta = 0, 
                            patience = 10,
                            verbose = 1,
                            restore_best_weights = True)

        #reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, 
        #                           verbose=1, mode='max', min_lr=0.00001)
        reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', #'loss',
                              factor = 0.5,
                              patience = 2,
                              verbose = 1,
                              min_lr=0.000001,
                              min_delta = 0.0001)
        # Save the best model during the traning
        # loss,accuracy,val_loss,val_accuracy,lr
        check_name = OUTPUT_DIR +  'best_'+self.model_name+'.h5'
        print(check_name)
        checkpointer = ModelCheckpoint( check_name
                                        #,monitor='val_loss'
                                        #,mode="min"
                                        ,monitor='val_accuracy'
                                        ,mode="max"                                       
                                        ,verbose=1
                                        ,save_best_only=True)
                                        #,save_weights_only=True)

        #callbacks = [ checkpointer,lr_callback]        
            
        callbacks = [earlystopper, checkpointer, reduce_lr] #lr_callback] #
        # Train
        training = model.fit(self.generators.train_generator
                                ,epochs=epochs
                                ,steps_per_epoch=steps_per_epoch
                                ,validation_data=self.generators.val_generator
                                ,validation_steps=validation_steps
                                ,callbacks=callbacks
                                ,use_multiprocessing=False
                                ,shuffle=True
                                ,verbose=True)     
        
        # Get the best saved weights
        #model.load_weights(check_name)
        return training


In [ ]:

#model = trainer.create_model_3()  # Best ! 0.9219 Total params: 27,935,872  
#model = trainer.create_model_2()   # good ! 0.8750 Total params: 7,049,804  
#model = trainer.create_model_small() # worse 0.5625 Total params: 33,560,364
#model = trainer.create_model()  # bad delete; Total params: 33,682,380
#model = trainer.create_model("middle") # bad Total params: 3,017,928

#model = trainer.create_model("VGG16") # 0.7344  Total params: 14,720,844
#model = trainer.create_model("ResNet50V2") # 0.8906 Total params: 23,589,388
#model = trainer.create_model("InceptionResNetV2") # 0.8906 Total params: Total params: 54,355,180

#model = trainer.create_model("XceptionDenseNet121") # 0.9062 Total params: 27,935,872

# Create and train the model
trainer = ModelTrainer(generators)
#model = trainer.create_model("EfficientNetB7")   # Total params: 64,128,419
model = trainer.create_model("XceptionDenseNet121")
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


#f1 = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='macro')
#model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=1e-4),  metrics= [f1])


model.summary()

In [ ]:
start = time.time()
training=trainer.train(model, toy)
print("Trained Time : ", time.time() - start)
        
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

#tf.keras.models.save_model(model, './model/plant_model.hdf5')
#model.load_weights('../working/best_XceptionDenseNet121.h5')


#Epoch 1/100

#Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
#31/31 [==============================] - 62s 2s/step - loss: 2.2864 - accuracy: 0.2022 - val_loss: 2.2638 - val_accuracy: 0.3125

#model_2
#Epoch 00019: LearningRateScheduler reducing learning rate to 0.0001.
#31/31 [==============================] - 363s 12s/step - loss: 0.5241 - accuracy: 0.8358 - val_loss: 0.5352 - val_accuracy: 0.8750

#model_3
#Epoch 00029: LearningRateScheduler reducing learning rate to 1.9663676416000005e-05.
#31/31 [==============================] - 666s 21s/step - loss: 0.4607 - accuracy: 0.8552 - val_loss: 0.3860 - val_accuracy: 0.9219


#model_small
#Epoch 00036: LearningRateScheduler reducing learning rate to 1.2026619832316725e-05.
#31/31 [==============================] - 50s 2s/step - loss: 1.6877 - accuracy: 0.3644 - val_loss: 1.3935 - val_accuracy: 0.5625

#model = trainer.create_model_1() # Total params: 20,886,068
#Epoch 00033: LearningRateScheduler reducing learning rate to 1.3958241859993605e-05.
#31/31 [==============================] - 329s 11s/step - loss: 0.5072 - accuracy: 0.8469 - val_loss: 0.4209 - val_accuracy: 0.9062

#model = trainer.create_model("VGG16") # Total params: 14,720,844
#31/31 [==============================] - 361s 12s/step - loss: 1.1590 - accuracy: 0.6339 - val_loss: 0.8088 - val_accuracy: 0.7344

#model = trainer.create_model("ResNet50V2") #Total params: 23,589,388
#Epoch 00025: LearningRateScheduler reducing learning rate to 3.359296000000001e-05.
#31/31 [==============================] - 257s 8s/step - loss: 0.7078 - accuracy: 0.7953 - val_loss: 0.4911 - val_accuracy: 0.8906

#model = trainer.create_model("InceptionResNetV2") # 0.8906 Total params: Total params: 54,355,180
#Epoch 00025: LearningRateScheduler reducing learning rate to 3.359296000000001e-05.
#31/31 [==============================] - 257s 8s/step - loss: 0.7078 - accuracy: 0.7953 - val_loss: 0.4911 - val_accuracy: 0.8906

#model = trainer.create_model("XceptionDenseNet121") # 0.9062 Total params: 27,935,872
#Epoch 00025: LearningRateScheduler reducing learning rate to 3.359296000000001e-05.
#31/31 [==============================] - 676s 22s/step - loss: 0.5947 - accuracy: 0.8298 - val_loss: 0.4676 - val_accuracy: 0.9062

# 5. Evaluate trained model
Also put all evaluation code into one class for better code modularity.

In [ ]:
# Saving model history
#pd.DataFrame(training.history).to_csv('ModelHistory.csv')

In [ ]:
class Evaluator:
    """
    Evaluaion :predict on test data (not submission data from test folder)
    and print reports, plot results etc.
    """
     
    def __init__(self, model, training, generator, y_true):
        self.training = training
        self.generator = generator
        # predict the data
        steps=5
        preds = model.predict(self.generator, steps=steps)
        print(preds)
        preds = preds.tolist()

        ####

        indices = []
        for pred in preds:
            temp = []
            for category in pred:
                if category>=0.3:
                    temp.append(pred.index(category))
            if temp!=[]:
                indices.append(temp)
            else:
                temp.append(np.argmax(pred))
                indices.append(temp)

        print(indices)
        


        testlabels = []


        for image in indices:
            temp = []
            for i in image:
                temp.append(str(class_labels[i]))
            testlabels.append(' '.join(temp))

        print(testlabels)

                
        
        self.y_pred = testlabels
        self.y_true = y_true[:len(self.y_pred)]
        
        #print(self.y_pred_raw[0])
        #print(self.y_pred)
        #print(len(self.y_pred))
        #print(self.y_true[0], y_true[0])
        #print(y_true.shape, self.y_true.shape)
        #print(y_true)
        #print(y_true[:len(self.y_pred)] )
    
    def plot_history(self): 
        fig, ax1 = plt.subplots(figsize=(15, 8))
        history = self.training

        ax1.plot(history.history['loss'], 'r', label="training loss ({:.6f})".format(history.history['loss'][-1]))
        ax1.plot(history.history['val_loss'], 'r--', label="validation loss ({:.6f})".format(history.history['val_loss'][-1]))
        ax1.grid(True)
        ax1.set_xlabel('iteration')
        ax1.legend(loc="best", fontsize=9)    
        ax1.set_ylabel('loss', color='r')
        ax1.tick_params('y', colors='r')

        accname = "accuracy"
        if accname in history.history:
            ax2 = ax1.twinx()

            ax2.plot(history.history[accname], 'b', label="training acc ({:.4f})".format(history.history[accname][-1]))
            ax2.plot(history.history['val_' + accname], 'b--', label="validation acc ({:.4f})".format(history.history['val_' + accname][-1]))

            ax2.legend(loc="best", fontsize=12)
            ax2.set_ylabel('acc', color='b')        
            ax2.tick_params('y', colors='b')
        
    """
    Accuracy, evaluation
    """
    def plot_history1(self):
        """
        Plot training history
        """
        ## Trained model analysis and evaluation
        f, ax = plt.subplots(1,2, figsize=(15,5))
        ax[0].plot(self.training.history['loss'], label="Loss")
        ax[0].plot(self.training.history['val_loss'], label="Validation loss")
        ax[0].set_title('Loss')
        ax[0].set_xlabel('Epoch')
        ax[0].set_ylabel('Loss')
        ax[0].legend()

        # Accuracy
        ax[1].plot(self.training.history['accuracy'], label="Accuracy")
        ax[1].plot(self.training.history['val_accuracy'], label="Validation accuracy")
        ax[1].set_title('Accuracy')
        ax[1].set_xlabel('Epoch')
        ax[1].set_ylabel('Accuracy')
        ax[1].legend()
        plt.tight_layout()
        plt.show()
    
    def plot_roc(self):
        #y_pred_keras = model.predict_generator(test_gen, steps=len(df_val), verbose=1)
        # Calculate roc
        fpr_keras, tpr_keras, thresholds_keras = roc_curve(self.y_true, self.y_pred)
        auc_keras = auc(fpr_keras, tpr_keras)
        
        plt.figure(1)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.plot(fpr_keras, tpr_keras, label='area = {:.3f}'.format(auc_keras))
        plt.xlabel('False positive rate')
        plt.ylabel('True positive rate')
        plt.title('ROC curve')
        plt.legend(loc='best')
        plt.show()
        
    def print_report(self):
        """
        Predict and evaluate using ground truth from labels
        Test generator did not shuffle 
        and we can use true labels for comparison
        """
        #Print classification report
        #print(self.y_true.shape, self.y_pred.shape)
        #print(self.y_true[0], self.y_pred[0])
        #print(self.generator.classes)
        #print(metrics.classification_report(self.y_true, self.y_pred))
        
        #class_labels = self.generator.class_indices
        #print(class_labels)
        #class_labels = {v: k for k, v in class_labels.items()}
        #classes = list(class_labels.values())
        #y_pred = []
        #for item in self.y_pred:
        #    y_pred.append(classes[item])
            
        
        #self.y_pred = y_pred 
        print(metrics.classification_report(self.y_true, self.y_pred))
        

        
        #print('Confusion Matrix')
        #print(confusion_matrix(validation_generator.classes, y_pred))
        #print('Classification Report')
        #target_names = list(class_labels.values())
        #print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

# Create evaluator instance
evaluator = Evaluator(model, training, generators.test_generator, test_df.labels.values)

# Draw accuracy and loss charts
evaluator.plot_history()

# ROC curve
#evaluator.plot_roc()

# Classification report
#evaluator.print_report()

#print(self.y_pred_raw[0])  # 12 proba
#print(self.y_pred) # 160 pred result
#print(len(self.y_pred)) #160
#print(self.y_true[0], y_true[0]) #frog_eye_leaf_spot frog_eye_leaf_spot
#print(y_true.shape) #250
#print(y_true) # 250 labels string
#print(y_true[:len(self.y_pred)] )

In [ ]:
#evaluator.y_true

# 5. Submission
>Use ImageDataGenerator to reduce memory usage. My initial idea was to use generator.flow_from_directory for input/test folder but it didn't work for me. Quick fix is to use generator.flow_from_dataframe on dataframe with list of filenames.

In [ ]:
class Submitter:
    """
    Predict and submit
    """
    def __init__(self, model, img_size):
        self.model = model
        batch_size=BATCH_SIZE
        print("Initializing submitter")
        #Submission generator
        # flow_from_directory for input/test didn't work for me, so quick fix is to use flow_from_dataframe with list of files
        # Load list of files from test folder into dataframe
        self.test_files_df=pd.DataFrame()
        TEST_DIR = INPUT_DIR + 'test_images/'
        self.test_files_df['image']=os.listdir(TEST_DIR)
        print("Loaded test files list")
        
        
        # Create generator in it
        #self.generator=ImageDataGenerator(rescale=1./255.).flow_from_dataframe(
        _test_datagen=ImageDataGenerator(rescale=1./255.)
        self.generator = _test_datagen.flow_from_dataframe(
                    dataframe=self.test_files_df,
                    directory=TEST_DIR,
                    x_col="image",
                    y_col=None,
                    #has_ext=True,
                    class_mode=None,
                    batch_size=batch_size,
                    seed=42,
                    shuffle=False,
                    target_size=img_size)    
        
        print('Submission generator created')    


    def predict_for_submit(self):
        """
        Predict submission test data and form dataframe to submit
        """
        print("Forming submission dataframe...")
        # Predict
        #y_pred = self.model.predict(self.generator)
        #y_pred = np.argmax(y_pred, axis=1)
        #print(y_pred)
        
        
        #y_pred_str = []
        #for item in y_pred:
        #    y_pred_str.append(classes[item])
        
        
        
        steps=5
        preds = self.model.predict(self.generator, steps=steps)
        print(preds)
        preds = preds.tolist()

        ####

        indices = []
        for pred in preds:
            temp = []
            for category in pred:
                if category>=0.3:
                    temp.append(pred.index(category))
            if temp!=[]:
                indices.append(temp)
            else:
                temp.append(np.argmax(pred))
                indices.append(temp)

        print(indices)
        


        testlabels = []


        for image in indices:
            temp = []
            for i in image:
                temp.append(str(class_labels[i]))
            testlabels.append(' '.join(temp))

        print(testlabels)
        
        
        self.test_files_df['labels'] = testlabels #y_pred_str
        # Write to csv
        self.test_files_df.to_csv('./submission.csv', index=False)
        print("Submission completed: written submission.csv")
        return self.test_files_df

In [ ]:

if not toy:
    # Get dataframe for submission
    submitter = Submitter(model, IMAGE_SIZE)
    submission_df = submitter.predict_for_submit()     
    submission_df.head()
else:
    submission_df = pd.DataFrame()
    print("Do not submit in toy mode")